In [1]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#list of words to filter out of statements
stop_words = set(stopwords.words('english'))

Helper functions to clean and generate sentiment analysis

In [2]:
from textblob import TextBlob
import ast
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import LancasterStemmer

# stemmer to turn words into their roots
lan=LancasterStemmer()

def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+|ftp\S+", '', tweet, flags=re.MULTILINE)
    tweet = re.sub('(@Tesla)','Tesla',tweet)
    tweet = re.sub('(&amp)','',tweet)
    tweet = re.sub(r'\@\w+|\#','', tweet)
    tweet = re.sub('[^\w ]','',tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet_tokens = word_tokenize(tweet,preserve_line=True)
    filtered_words = [lan.stem(w) for w in tweet_tokens if not w in stop_words]

    return " ".join(filtered_words)

def avg(data):
    data = ast.literal_eval(data)
    sum = 0
    for item in data:
        sum += item
    return sum/len(data)

def getSIA(text):
    return SentimentIntensityAnalyzer().polarity_scores(text)

def vader(row):
    tweets = ast.literal_eval(row['Tweets'])
    pos= []
    neg= []
    neu= []
    for tweet in tweets:
        SIA = getSIA(preprocess_tweet_text(tweet))
        pos.append(SIA['pos'])
        neg.append(SIA['neu'])
        neu.append(SIA['neg'])
    return np.average(pos), np.average(neg), np.average(neu)

Read csv and turn data call helper functions

In [3]:
df=pd.read_csv("results.csv")

df['pos'] = df.apply(vader,axis=1)
df[['pos', 'neg', 'neu']] = pd.DataFrame(df['pos'].tolist(), index=df.index)
df['Likes'] = df['Likes'].apply(avg)
df['label'] = (df['Dif'] > 0) * 1

Filter columns for features and label

In [48]:
keep = ['pos', 'neu','neg','label', 'Likes']
filtered = df[keep]
print(filtered.columns)

Index(['pos', 'neu', 'neg', 'label', 'Likes'], dtype='object')


Split data

In [49]:
from sklearn.model_selection import train_test_split

X = filtered.loc[ : , filtered.columns != 'label']
y = filtered['label']

x_train, x_test, y_train, y_test =  train_test_split(X , y, test_size = 0.2, random_state = 0)

Train model, and generate the scores

In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, f1_score

model = RandomForestClassifier(n_estimators=190, random_state=36).fit(x_train,y_train)
y_pred = model.predict(x_test)
f1 = f1_score(y_test,y_pred)

print("Accuracy is:", accuracy_score(y_test, y_pred))

print("F-score is:", f1_score(y_test, y_pred))

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print("Area Under Curve:", auc(fpr, tpr))

classificationReport = classification_report(y_test, y_pred, target_names=['Price Drop', 'Price Rise'])
print("\nClassification Report:\n", classificationReport)

Accuracy is: 0.6143497757847534
F-score is: 0.6814814814814815
Area Under Curve: 0.6017799352750809

Classification Report:
               precision    recall  f1-score   support

  Price Drop       0.62      0.44      0.51       103
  Price Rise       0.61      0.77      0.68       120

    accuracy                           0.61       223
   macro avg       0.61      0.60      0.60       223
weighted avg       0.61      0.61      0.60       223

